Needed:

* **Queue** = "solo/duo"

  - queue_type: "Ranked Solo/Duo"

   -  highestTierAchieved: RANKED_SOLO_5x5

* ranked match
* several players
* different players' ranks --> **tier**


In [ ]:
import requests
import time
import random
import json
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
SLEEP_DURATION = 10
def get_leagues(platform, api):
  """
  Returns a list of 987 summonerNames from all ranks in the given platform, according to the following rank distribution:

  C: 0.010%, GM: 0.025%, M: 0.20%, D: 1.6%, P: 12%, G: 29%, S: 36%, B: 18%, I: 1.7%
  retrieved form: https://www.leagueofgraphs.com/rankings/rank-distribution

  Inputs:
  - platform: One of these values ['BR1', 'eun1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'RU', 'TR1']
  - api: Your api key.

  Output: 
  - summonerName: List of summoner Names in the given platform.
  """

  leagues = ['challengerleagues', 'grandmasterleagues', 'masterleagues', 'DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON']
  
  # Challenger (0.1 of 1000)
  url_challenger = 'https://'+ platform + '.api.riotgames.com/lol/league/v4/'+ leagues[0]  + '/by-queue/RANKED_SOLO_5x5?api_key=' + api
  response = requests.get(url_challenger)
  lis = response.json()['entries']
  summonerName_c = [ lis[0]['summonerName'] ]

  # GrandMaster (0.25 of 1000)
  url_grandmaster = 'https://'+ platform + '.api.riotgames.com/lol/league/v4/'+ leagues[1]  + '/by-queue/RANKED_SOLO_5x5?api_key=' + api
  response = requests.get(url_grandmaster)
  lis = response.json()['entries']
  summonerName_gm = [ lis[0]['summonerName'] ]

  # Master (2 of 1000)
  url_master = 'https://'+ platform + '.api.riotgames.com/lol/league/v4/'+ leagues[2]  + '/by-queue/RANKED_SOLO_5x5?api_key=' + api
  response = requests.get(url_master)
  lis = response.json()['entries']
  summonerName_m = [sub['summonerName'] for sub in lis[0:2]]

  # Diamond (16 of 1000)
  url_diamond = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[3] + '/IV?page=1&api_key=' + api 
  response = requests.get(url_diamond)
  lis = response.json()
  summonerName_d = [dic['summonerName'] for dic in lis[0:16]]

  # Platinum (120 of 1000)
  url_platinum = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[4] + '/IV?page=1&api_key=' + api 
  response = requests.get(url_platinum)
  lis = response.json()
  summonerName_p = [dic['summonerName'] for dic in lis[0:120]]

  # Gold (290 of 1000)
  url_gold = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[5] + '/IV?page=1&api_key=' + api 
  response = requests.get(url_gold)
  lis = response.json()
  summonerName_gd = [dic['summonerName'] for dic in lis]  #205
  url_gold = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[5] + '/III?page=1&api_key=' + api 
  response = requests.get(url_gold)
  lis = response.json()
  summonerName_gd2 = [dic['summonerName'] for dic in lis[0:85]]
  
  # Silver (360 of 1000)
  url_silver = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[6] + '/IV?page=1&api_key=' + api 
  response = requests.get(url_silver)
  lis = response.json()
  summonerName_s = [dic['summonerName'] for dic in lis]    #205
  url_silver = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[6] + '/III?page=1&api_key=' + api 
  response = requests.get(url_silver)
  lis = response.json()
  summonerName_s2 = [dic['summonerName'] for dic in lis[0:155]]

  # Bronze  (180 of 1000)
  url_bronze = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[7] + '/IV?page=1&api_key=' + api 
  response = requests.get(url_bronze)
  lis = response.json()
  summonerName_b = [dic['summonerName'] for dic in lis[0:180]]

  # Iron (17 of 1000)
  url_iron = 'https://' + platform.lower() +'.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/' + leagues[8] + '/IV?page=1&api_key=' + api 
  response = requests.get(url_iron)
  lis = response.json()
  summonerName_i = [dic['summonerName'] for dic in lis[0:17]]


  summonerName = summonerName_c + summonerName_gm + summonerName_m + summonerName_d + summonerName_p + summonerName_gd + summonerName_gd2 + summonerName_s + summonerName_s2 + summonerName_b + summonerName_i

  return summonerName


def get_puuid(api, summonerNames, process, platform='BR1'):
  """
  Returns a list of the puuids of each summoner of the given list of summoner names in a given platform.

  Inputs:
  - platform: One of these values ['BR1', 'eun1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'RU', 'TR1']
  - api: Your api key.
  - summonerNames: A list of summoner names in a given platform.

  Output:
  - puuid_list: A list of puuids of the summoners.
  """

  puuid_list = []
  c = 0
  for name in summonerNames:

    url = 'https://' + platform.lower() + '.api.riotgames.com/lol/summoner/v4/summoners/by-name/'  + name + '?api_key=' + api
    response = requests.get(url).json()
    if c % 100 == 0:
      print("p{} getting player {} puuid".format(process, c))
    c += 1
    # If any error happens, skip that summoner
    if response.get('status'):
      print("p{} Error summoner '{}' {}".format(process, name, response['status']['message']))
      if response['status']['message'] == "Rate limit exceeded":
        while response['status']['message'] == "Rate limit exceeded":
          print("Sleeping zzzzzz")
          time.sleep(SLEEP_DURATION)
          response = requests.get(url).json()
          if not response.get('status'):
            break
      else:
        continue
    puuid = response['puuid']
    puuid_list.append(puuid)
    print(len(puuid_list))
  return puuid_list



def get_matchId(api, puuids, process, region='americas'):
  """
  Returns a list of 20 match ids using the given list of puuids of summoners in a specific region.

  Inputs:
  - region: 'americas', 'asia' or 'europe'.
  - api: Your api key.
  - puuids: A list of puuids of summoners.

  Output:
  - matchId_list: A list of matches played by these summoners. 100 matches are retrieved for each summoner.
  
  """

  matchId_list = []
  c = 0
  for puuid in puuids:
    # Get the ids of 20 matches played by that summoner with the puuid
    url = 'https://'+ region +'.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid +'/ids?start=0&count=20&api_key=' + api
    response = requests.get(url).json()
    if c % 100 == 0:
      print("p{} getting match {} id".format(process, c))
    c += 1
    # If rate limit is exceeded
    if isinstance(response, dict):
      print("p{} Error getting matchid {}".format(process, response['status']['message']))
      if response['status']['message'] == "Rate limit exceeded":
        while response['status']['message'] == "Rate limit exceeded":
          print("Sleeping zzzzzz")
          time.sleep(SLEEP_DURATION)
          response = requests.get(url).json()
          if not isinstance(response, dict):
            break
      else:
        continue

    matchId_list.append(response[0])

  return matchId_list



def get_matches(api, matchIds, process, region='americas'):
  """
  Get the matches from the given matchIds list.

  Inputs: 
  - region: 'americas', 'asia' or 'europe'.
  - api: Your api key.
  - matchIds: A list of matches played by summoners. 

  Output:
  - matches: A dictionary containing 10000 matches.
  """
  matches = []
  
  i = 0
  c = 0
  for matchId in matchIds:
    time.sleep(0.01)
    url = 'https://' + region + '.api.riotgames.com/lol/match/v5/matches/' + matchId + '?api_key=' + api
    response = requests.get(url).json()
    if c % 100 == 0:
      print("p{} getting match {}".format(process, c))
    c += 1
    # If rate limit is exceeded
    if response.get('status'):
      print("p{} Error getting match {}".format(process, response['status']['message']))
      if response['status']['message'] == "Rate limit exceeded":
        while response['status']['message'] == "Rate limit exceeded":
          print("Sleeping zzzzzz")
          time.sleep(SLEEP_DURATION)
          response = requests.get(url).json()
          if not response.get('status'):
            break
      else:
        continue
    
    i += 1
    # print("Step = {}".format(i))
    matches.append(response)

  return matches


# This function wraps all of the above functions
def getMatchesFromPlatform(region, platform, api):

  """
  Get 100,000 matches from the given platform, saves it in a file and dowlnoads it.

  Inputs:
  - region: 'americas', 'asia' or 'europe'.
  - platform: One of these values ['BR1', 'eun1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'RU', 'TR1']
  - api: Your api key.

   Output:
  - matches: A dictionary containing 1000 matches.

  """
  summonerNames = get_leagues(platform, api)
  puuids = get_puuid(platform, api, summonerNames)
  match_ids = get_matchId(region, api, puuids)
  matches = get_matches(region, api, match_ids)

  with open('matches', 'w') as f:
    json.dump(matches, f)

  files.download("matches")

In [ ]:
from collections import deque

def bfs(puuids, api, depth, process, platform='BR1', region='americas', children=9):
  '''Breadth-first tree search implementation.'''
  current_depth = 0
  while current_depth < depth:
    frontier = deque(puuids)
    puuids = []
    node_count = 0
    while frontier:
      try:
        current_puuid = frontier.pop()      
        print("depth={}, node={}".format(current_depth, node_count))
        match_id = get_matchId(api, [current_puuid], process)[0]
        match = get_matches(api, [match_id], process)[0]
        participants = match['metadata']['participants']
        participants.remove(current_puuid)
        participants = participants[:children]
        puuids.extend(participants)
        node_count += 1
      except:
        pass
    current_depth += 1
  return puuids

In [ ]:
!gdown --id 15e67JCKOFUz36LVuvXoV7m9CKvHtOH_m
!gdown --id 1j1UHaV0UPJ5PzdXIQgKDx-b26tvL7aH6

Downloading...
From: https://drive.google.com/uc?id=15e67JCKOFUz36LVuvXoV7m9CKvHtOH_m
To: /content/api.txt
100% 1.89k/1.89k [00:00<00:00, 4.21MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j1UHaV0UPJ5PzdXIQgKDx-b26tvL7aH6
To: /content/summoners.txt
100% 11.2k/11.2k [00:00<00:00, 9.54MB/s]


In [ ]:
apis = []
with open("api.txt", "r") as file:
  for line in file:
    line = line.strip().split('\t')
    if len(line) == 1:
      apis.append(line[0])
    else:
      apis.append(line[1])

In [ ]:
summonerNames = []
with open("summoners.txt", "r") as file:
  for line in file:
    summonerNames.append(line.strip())

In [ ]:
def wrapper(api, summonerNames, output):
  process = output
  puuids = get_puuid(api, summonerNames, process)
  puuids = bfs(puuids, api, 3, process, children=5)
  match_ids = get_matchId(api, puuids, process)
  matches = get_matches(api, match_ids, process)
  with open('/content/gdrive/MyDrive/bd_mini2/part'+str(output)+'.json', 'w') as f:
    json.dump(matches, f)

In [ ]:
from multiprocessing import Process

processes = []
c = 0
l = [0, 1, 5, 6, 15, 21, 29]
for i in range (0, 987, 33):
  if c in l:
    p = Process(target=wrapper, args=(apis[c], summonerNames[i:i+33], c, ))
    p.start()
    processes.append(p)
  c += 1
for p in processes:
  p.join()

Streaming output truncated to the last 5000 lines.
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
p0 getting match 1700 id
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
p0 Error getting matchid Rate limit exceeded
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
Sleeping zzzzzz
p6 getting match 2100 id
p15 getting match 2100 id
Sleeping zzzzzz
p29 getting 